# Imports

In [1]:
import os
os.system('pip install tasknet pygrove xpflow setGPU pandit imblearn')

0

In [2]:
import pandit as pd
import numpy as np
import tasknet as tn
from tqdm.notebook import tqdm
from dataclasses import dataclass
tqdm.pandas()
import random
from datasets import DatasetDict
import datasets
from torch.utils.data import Dataset
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoTokenizer
import inspect
import torch
import transformers
from types import MethodType
os.environ["WANDB_DISABLED"] = "true"
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import tensorflow as tf
from collections import Counter
import imblearn
import funcy as fc

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
from huggingface_hub import login
from  credentials import huggingfacehub #create credentials.py file with huggingfacehub variable and API key as value.
login(huggingfacehub)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to C:\Users\Quent\.cache\huggingface\token
Login successful


# Loading Data

## Datasets

In [283]:
url = 'https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_categorized.json'
df_reddit= pd.read_json('https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_categorized.json')
del df_reddit["Unnamed: 0"]

In [284]:
df = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/WVMCQ7/WVMCQ7-Normalized_labels_percentage_wise.json")
df = df[df["demographic_topic"]!= "politic"] 

## Demographic libraries

In [285]:
stop_words = ["able","about","above","abroad","according","accordingly","across","actually","adj","after","afterwards","again","against","ago","ahead","ain't","all","allow","allows","almost","alone","along","alongside","already","also","although","always","am","amid","amidst","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","a's","aside","ask","asking","associated","at","available","away","awfully","back","backward","backwards","be","became","because","become","becomes","becoming","been","before","beforehand","begin","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","came","can","cannot","cant","can't","caption","cause","causes","certain","certainly","changes","clearly","c'mon","co","co.","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","c's","currently","dare","daren't","definitely","described","despite","did","didn't","different","directly","do","does","doesn't","doing","done","don't","down","downwards","during","each","edu","eg","eight","eighty","either","else","elsewhere","end","ending","enough","entirely","especially","et","etc","even","ever","evermore","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","fairly","far","farther","few","fewer","fifth","first","five","followed","following","follows","for","forever","former","formerly","forth","forward","found","four","from","further","furthermore","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","had","hadn't","half","happens","hardly","has","hasn't","have","haven't","having","he","he'd","he'll","hello","help","hence","her","here","hereafter","hereby","herein","here's","hereupon","hers","herself","he's","hi","him","himself","his","hither","hopefully","how","howbeit","however","hundred","i'd","ie","if","ignored","i'll","i'm","immediate","in","inasmuch","inc","inc.","indeed","indicate","indicated","indicates","inner","inside","insofar","instead","into","inward","is","isn't","it","it'd","it'll","its","it's","itself","i've","just","k","keep","keeps","kept","know","known","knows","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","likewise","little","look","looking","looks","low","lower","ltd","made","mainly","make","makes","many","may","maybe","mayn't","me","mean","meantime","meanwhile","merely","might","mightn't","mine","minus","miss","more","moreover","most","mostly","mr","mrs","much","must","mustn't","my","myself","name","namely","nd","near","nearly","necessary","need","needn't","needs","neither","never","neverf","neverless","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","no-one","nor","normally","not","nothing","notwithstanding","novel","now","nowhere","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","one's","only","onto","opposite","or","other","others","otherwise","ought","oughtn't","our","ours","ourselves","out","outside","over","overall","own","particular","particularly","past","per","perhaps","placed","please","plus","possible","presumably","probably","provided","provides","que","quite","qv","rather","rd","re","really","reasonably","recent","recently","regarding","regardless","regards","relatively","respectively","right","round","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","shan't","she","she'd","she'll","she's","should","shouldn't","since","six","so","some","somebody","someday","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that's","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","there'd","therefore","therein","there'll","there're","theres","there's","thereupon","there've","these","they","they'd","they'll","they're","they've","thing","things","think","third","thirty","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","till","to","together","too","took","toward","towards","tried","tries","truly","try","trying","t's","twice","two","un","under","underneath","undoing","unfortunately","unless","unlike","unlikely","until","unto","up","upon","upwards","us","use","used","useful","uses","using","usually","v","value","various","versus","very","via","viz","vs","want","wants","was","wasn't","way","we","we'd","welcome","well","we'll","went","were","we're","weren't","we've","what","whatever","what'll","what's","what've","when","whence","whenever","where","whereafter","whereas","whereby","wherein","where's","whereupon","wherever","whether","which","whichever","while","whilst","whither","who","who'd","whoever","whole","who'll","whom","whomever","who's","whose","why","will","willing","wish","with","within","without","wonder","won't","would","wouldn't","yes","yet","you","you'd","you'll","your","you're","yours","yourself","yourselves","you've","zero","a","how's","i","when's","why's","b","c","d","e","f","g","h","j","l","m","n","o","p","q","r","s","t","u","uucp","w","x","y","z","I","www","amount","bill","bottom","call","computer","con","couldnt","cry","de","describe","detail","due","eleven","empty","fifteen","fifty","fill","find","fire","forty","front","full","give","hasnt","herse","himse","interest","itse”","mill","move","myse”","part","put","show","side","sincere","sixty","system","ten","thick","thin","top","twelve","twenty","abst","accordance","act","added","adopted","affected","affecting","affects","ah","announce","anymore","apparently","approximately","aren","arent","arise","auth","beginning","beginnings","begins","biol","briefly","ca","date","ed","effect","et-al","ff","fix","gave","giving","heres","hes","hid","home","id","im","immediately","importance","important","index","information","invention","itd","keys","kg","km","largely","lets","line","'ll","means","mg","million","ml","mug","na","nay","necessarily","nos","noted","obtain","obtained","omitted","ord","owing","page","pages","poorly","possibly","potentially","pp","predominantly","present","previously","primarily","promptly","proud","quickly","ran","readily","ref","refs","related","research","resulted","resulting","results","run","sec","section","shed","shes","showed","shown","showns","shows","significant","significantly","similar","similarly","slightly","somethan","specifically","state","states","stop","strongly","substantially","successfully","sufficiently","suggest","thered","thereof","therere","thereto","theyd","theyre","thou","thoughh","thousand","throug","til","tip","ts","ups","usefully","usefulness","'ve","vol","vols","wed","whats","wheres","whim","whod","whos","widely","words","world","youd","youre"]
import requests
import re

r = requests.get("https://raw.githubusercontent.com/dariusk/corpora/master/data/religion/religions.json")
religions = r.json()
beliefs = []
for major in [item for item in religions.values()]:
    majors = [m for m in major]
    beliefs.extend(majors)
    try:
        for minor in major.values():
            for religion in minor:
                beliefs.append(religion)
    except:
        next
r = requests.get("https://raw.githubusercontent.com/Imagin-io/country-nationality-list/master/countries.json")
dictionary = r.json()
df_dictionary = pd.DataFrame(dictionary)
def nation(item):
    item["en_short_name"] = ""+item["en_short_name"].lower()+""
    item["alpha_2_code"] = ""+item["alpha_2_code"].lower()+""
    item["alpha_3_code"] = ""+item["alpha_3_code"].lower()+""
    nationality = re.split(', | or',item["nationality"].lower())
    item["nationality"] = nationality
    return item
df_dictionary=df_dictionary.apply(nation,axis=1)
nationalities = []
for item in df_dictionary["nationality"].values:
    for label in item:
        nationalities.append(label)

countries = []
countries.extend(df_dictionary["en_short_name"].values)
countries.extend(df_dictionary["alpha_2_code"].values)
countries.extend(df_dictionary["alpha_3_code"].values)

# Data selection and processing

## Preprocessing Methods

In [286]:
import importlib
importlib.reload(tn)

<module 'tasknet' from 'c:\\Users\\Quent\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\tasknet\\__init__.py'>

In [287]:
@dataclass
class DataCollatorForPolls:
    tokenizer: None=None
    tokenizer_kwargs: None=None
    def __call__(self, features):
        print('datacollator')
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        
        flattened_features = sum(flattened_features, [])
        pad_args=inspect.signature(self.tokenizer.pad).parameters.keys()
        batch = self.tokenizer.pad(flattened_features, **fc.project(self.tokenizer_kwargs,pad_args))
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.stack(labels).float() #### Allows one value per choice
        print(batch)
        return batch

In [288]:
#reddit
def get_demographics(x):
    parenthesized=[s[s.find("(")+1:s.rfind(")")]  for s in x.options.keys()]
    parenthesized=[s for s in parenthesized if s]
    parenthesized_count=pd.Series.value_counts(parenthesized)
    return list(parenthesized_count[parenthesized_count>1].index)

def split_questions(x):
    demographics=get_demographics(x)

    l=[]
    for d in demographics:
        l+=[fc.project(x.options, [x for x in x.options if f'({d})' in x])] #if row has multiple demographics its repeated in dataset?
    if l:
        x.options=l
        x['split']=True
    else:
        x.options=[x.options]
        x['split']=False
    return x

def extract_demographic(x):
    demographics = get_demographics(x)
    x['demographic']=''
    if not demographics:
        return x
    if len(demographics)>1:
        print('failure:',demographics, x.options)
        return x
    demographic=demographics[0]
    x['options'] = {k.replace(f'({demographic})',''):v for (k,v) in x.options.items()}
    x['demographic']=demographic
    return x

num_choices=6
choices_names=[f'choice_{i}' for i in range(num_choices)]
mode='proba'

def preprocess(x):
    choices = (list(x.options.keys())+['']*num_choices)[:num_choices]
    for i,n in enumerate(choices_names):
        x[n]=choices[i]
    scores = np.array(list(x.options.values())+[-1]*num_choices)[:num_choices]*1.0
    if mode=='binary':
        x['label'] = np.random.choice(np.flatnonzero(scores == scores.max()))
    else:
        scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
        x['label'] = scores
    del x['options']
    x['task']=0
    return x

def remove_result(x):
  newDict={}
  for key,value in x.items():
    if "result" not in key.lower():
      newDict[key] = value
  return newDict

In [289]:
def choices(x):
    choices = (x["options"]+['']*num_choices)[:num_choices]
    scores = np.array(list(x.labels)+[-1]*num_choices)[:num_choices]*1.0
    for i,n in enumerate(choices_names):
        x[n]=choices[i]  
    x["labels"] = scores
    return x

def filter_data(x):
    options = x["options"]
    if(1 < len(options) <7):
        return x

def apply_question_template(df):
  #we need to take into account that the input to the model is Demographic + Question + Answer
  # we thus need to create a question template that seems natural
  # This template is only necessary if the input is not already in a question format
  demographic = df["demographic"]
  category = df["demographic_category"]
  subject = df["input"]
  result = any(item in subject for item in ["?","How","Why","Where","When","Who","What"])
  sentence = f"{demographic}: {subject} ?".lower()
  if(not result):
    if(category == "age"):
        sentence = f"{demographic} years old: which of these options would you take in regard to the subject '{subject}'?".lower()  
    elif(category == "married"):
        sentence = f"relationship satus {demographic}: which of these options would you take in regard to the subject '{subject}'?".lower()
    else:
        sentence = f"{demographic}: which of these options would you take in regard to the subject '{subject}'?".lower()  
  df['question'] = sentence
  return df

def get_split(x,train,validation):
    """parameters are wished percentage of dataset to be given to a split"""
    rnd=random.Random(x).random()
    if rnd<train:
        return 'train'
    if rnd<(train + validation):
        return 'validation'
    else:
        return 'test'
        
def to_year(x):
    x["created"] = pd.to_datetime(x["created"], unit='s').year
    return x

def shuffle_options(x):
    options = x["options"]
    labels = x["labels"]
    tupples = list(zip(options,labels))
    random.shuffle(tupples)
    x["options"] = [k for (k,_) in tupples]
    x["labels"] = [v for (_,v) in tupples]
    return x

## RedditPoll350k

In [290]:
df_reddit['inputs']=df_reddit.title
len(df_reddit)

312306

In [291]:
df_reddit['options']=df_reddit.options.map(lambda x:{k:v for k,v in x.items() if k!='Results'})
df_reddit['options'].sample(frac=1.0)#mix options ad random
df_reddit=df_reddit[df_reddit['options'].map(len)>1]
len(df_reddit)
df_reddit['n_answers']=df_reddit.options.map(dict.values).map(sum)
df_reddit=df_reddit[df_reddit.n_answers>20]
len(df_reddit)

307568

In [292]:
if 'dem' not in url:
    df_reddit=df_reddit.progress_apply(split_questions,axis=1).explode('options')
    df_reddit=df_reddit.progress_apply(extract_demographic,axis=1)
    df_reddit['inputs']=df_reddit.demographic+': '+df_reddit.title
df_reddit_demographics=df_reddit[df_reddit.demographic.map(len).map(bool)]
df_reddit_demographics.head(3), len(df_reddit_demographics)

  0%|          | 0/307568 [00:00<?, ?it/s]

  0%|          | 0/317731 [00:00<?, ?it/s]

failure: ['165 cm', "165 cm) - 6' (183 cm"] {'< 5\' 5" (165 cm), Male': 14, '< 5\' 5" (165 cm), Female': 34, '5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36}
failure: ["165 cm) - 6' (183 cm", '183 cm'] {'5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36, "> 6' (183 cm), Male": 65, "> 6' (183 cm), Female": 4}


(        id                                              title  \
 1   yvae7l  Do You Believe Your Congressional District is ...   
 1   yvae7l  Do You Believe Your Congressional District is ...   
 65  yuipdn                            Does body count matter?   
 
                     options     created               category  \
 1   {'Yes ': 35, 'No ': 17}  1668455219  news_&_social_concern   
 1     {'Yes ': 7, 'No ': 8}  1668455219  news_&_social_concern   
 65  {'Yes ': 70, 'No ': 52}  1668382874   diaries_&_daily_life   
 
                                                inputs  n_answers  split  \
 1   D: Do You Believe Your Congressional District ...        134   True   
 1   R: Do You Believe Your Congressional District ...        134   True   
 65                      Male: Does body count matter?        168   True   
 
    demographic  
 1            D  
 1            R  
 65        Male  ,
 20069)

In [293]:
counter = [x for x in Counter(df_reddit_demographics["demographic"]).keys() if Counter(df_reddit_demographics["demographic"])[x] >= 10]
df_reddit_demographics = df_reddit_demographics[df_reddit_demographics["demographic"].isin(counter)]

In [294]:
df_reddit_demographics_normalized = df_reddit_demographics.progress_apply(preprocess,axis=1)

  0%|          | 0/11561 [00:00<?, ?it/s]

C:\Users\Quent\AppData\Local\Temp\ipykernel_17516\3184463539.py:47: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_17516\3184463539.py:47: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_17516\3184463539.py:47: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_17516\3184463539.py:47: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
C:\Users\Quent\AppData\Local\Temp\ipykernel_17516\3184463539.py:47: RuntimeWarning: invalid value encountered in divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()


In [295]:
if mode=='binary':
    df_reddit_demographics_normalized=df_reddit_demographics_normalized[df_reddit_demographics_normalized.label<num_choices]
else:
    df_reddit_demographics_normalized=df_reddit_demographics_normalized[df_reddit_demographics_normalized.label.map(len)==num_choices]
len(df_reddit_demographics_normalized)

11561

In [296]:
df_reddit_demographics_normalized.dropna

<bound method DataFrame.dropna of             id                                              title     created  \
1       yvae7l  Do You Believe Your Congressional District is ...  1668455219   
65      yuipdn                            Does body count matter?  1668382874   
65      yuipdn                            Does body count matter?  1668382874   
66      yui5re                      are your nails long or short?  1668381574   
66      yui5re                      are your nails long or short?  1668381574   
...        ...                                                ...         ...   
307851  gq6x1r  Should women go to jail for false rape accusat...  1590393452   
307851  gq6x1r  Should women go to jail for false rape accusat...  1590393452   
308670  gkzer2                               Lying about your age  1589653023   
309958  gdk0w1                          Your preference in comics  1588625918   
311887  fsruvs                                 political ideology  15857121

In [297]:
df_reddit_demographics_normalized[df_reddit_demographics_normalized.label.progress_map(lambda x: any(np.isnan(a) for a in x))]
df_reddit_demographics_normalized['split']=df_reddit_demographics_normalized.title.map(lambda x: get_split(x,0.95,0.025))

  0%|          | 0/11561 [00:00<?, ?it/s]

### Normalize demographic

In [298]:
def normalize_nationality(x):
    male = ["man","guy"]
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in male:
      if any(token.replace(" ","").lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    x["demographic"] = label
    return x

def normalize_countries(x):
    male = ["man","guy","men"]
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in male:
      if any(token.replace(" ","").lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    x["demographic"] = label
    return x

def normalize_genders(x):
    label = x["demographic"].lower()
    female= ["f","female","woman","chick","girl"]
    male = ["m","male","man","guy","men"]
    if any(token.lower() in female for token in label.split()):
        if ("not" in label) | ("non" in label):
            label = "not female"
        else:
            label = "female"
    if any(token in male for token in label.split()):
        if("not" in label) | ("non" in label):
            label = "not male"
        else:
            label = "male"
    x["demographic"] = label
    return x

def normalize_politics(x):
    label = x["demographic"].lower()
    right = ["d","democratic","right","r",'right-leaning']
    left = ["republican","republicans","left","left-leaning"]
    if any(token.lower() in label.split(" ") for token in right):
        if("not" in label) | ("non" in label):
            label = "not democrat"
        else: 
            label = "democrat"
    elif any(token.lower() in label.split(" ") for token in left):
        if("not" in label) | ("non" in label):
            label = "not republican"
        else: 
            label = "republican"
    x["demographic"] = label
    return x

def remove_nan(x):
  labels = x["label"]
  if any(np.isnan(labels)):
      pass
  else:
      return x


In [299]:
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized.apply(normalize_nationality,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_genders,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_countries,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_politics,axis=1)
df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized_labels.progress_apply(remove_nan,axis=1)
df_reddit_demographics_normalized_labels.dropna()

  0%|          | 0/11561 [00:00<?, ?it/s]

,id,title,created,category,inputs,n_answers,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task
1,yvae7l,Do You Believe Your Congressional District is ...,1.668455e+09,news_&_social_concern,R: Do You Believe Your Congressional District ...,134.0,train,democrat,Yes,No,,,,,"[0.4666666666666667, 0.5333333333333333, -1.0,...",0.0
65,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,Male: Does body count matter?,168.0,train,male,Yes,No,,,,,"[0.5737704918032787, 0.4262295081967213, -1.0,...",0.0
65,yuipdn,Does body count matter?,1.668383e+09,diaries_&_daily_life,Female: Does body count matter?,168.0,train,female,Yes,No,,,,,"[0.2826086956521739, 0.717391304347826, -1.0, ...",0.0
66,yui5re,are your nails long or short?,1.668382e+09,fashion_&_style,Male: are your nails long or short?,631.0,test,male,Long,Short,,,,,"[0.06680584551148225, 0.9331941544885177, -1.0...",0.0
66,yui5re,are your nails long or short?,1.668382e+09,fashion_&_style,Female: are your nails long or short?,631.0,test,female,Long,Short,,,,,"[0.23026315789473684, 0.7697368421052632, -1.0...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307851,gq6x1r,Should women go to jail for false rape accusat...,1.590393e+09,news_&_social_concern,Female: Should women go to jail for false rape...,408.0,train,female,Yes,No,,,,,"[0.8681318681318682, 0.13186813186813187, -1.0...",0.0
307851,gq6x1r,Should women go to jail for false rape accusat...,1.590393e+09,news_&_social_concern,Other: Should women go to jail for false rape ...,408.0,train,other,Yes,No,,,,,"[0.8, 0.2, -1.0, -1.0, -1.0, -1.0]",0.0
308670,gkzer2,Lying about your age,1.589653e+09,diaries_&_daily_life,male: Lying about your age,165.0,train,male,Yes,No,,,,,"[0.42276422764227645, 0.5772357723577236, -1.0...",0.0
309958,gdk0w1,Your preference in comics,1.588626e+09,arts_&_culture,comment: Your preference in comics,97.0,train,comment,Multiple,Other,,,,,"[0.44, 0.56, -1.0, -1.0, -1.0, -1.0]",0.0


### Applying templates

#### Zeroshot for Masked Language Setting

In [300]:
def to_sentences(df):
    input_index = 1
    year_index = 2
    split_index = 4
    demographic_index= 5
    label_index = 13
    dataset_reddit_sentences = []
    for input in df.values:
        try:
            question_input = str(input[input_index])
            year = input[year_index]
            choices = choices = [input[7],input[8],input[9],input[10],input[11],input[12]]
            demographic = input[demographic_index]
            split = input[split_index]
            labels = input[label_index]
            
            for (choice,label) in zip(choices,labels):
                if(choice != ""):
                    sentence = f"In {year}, <mask> percent of {demographic} voted '{choice}' on the question '{question_input}'".lower()
                    dataset_reddit_sentences.append({"input": sentence,"subject":question_input,"choice":choice,"demographic":demographic,"year":year, "y": label,"split":split})
        except:
            pass
    return pd.DataFrame(dataset_reddit_sentences)

In [301]:
df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized_labels.progress_apply(to_year,axis=1)

  0%|          | 0/11561 [00:00<?, ?it/s]

In [302]:
df_reddit_demographics_normalized_sentences = to_sentences(df_reddit_demographics_normalized_labels)

In [303]:
#df_reddit_normalized_sentences = to_sentences(df_reddit_normalized)

#dataset_reddit_full=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_normalized_sentences[df_reddit_normalized_sentences['split']==k]) for k in {'train','test','validation'}})
dataset_wvs_zeroshot=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_demographics_normalized_sentences[df_reddit_demographics_normalized_sentences['split']==k]) for k in {'train','test','validation'}})


#### Fine-tuning and Zeroshot for MCQ setting

In [304]:
df_reddit_demographics_normalized_labels['inputs']=df_reddit_demographics_normalized_labels.demographic+': '+df_reddit_demographics_normalized_labels.title

In [305]:
newdf = df_reddit_demographics_normalized_labels.drop_duplicates(
  subset = ['title', 'demographic'],
  keep = 'last').reset_index(drop = True)

### Example

In [306]:
tokenizer =AutoTokenizer.from_pretrained("roberta-large", use_fast=True)

In [307]:
features = tokenizer("Hello, this one sentence!", "And this sentence goes with it.",truncation=True)
print(features)
tokenizer.decode(features["input_ids"])

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


'<s>Hello, this one sentence!</s></s>And this sentence goes with it.</s>'

### Reco

In [308]:
try:
    del df_reddit_demographics_normalized_labels['options']
except:
    pass
df_reddit_demographics_normalized_labels['labels']=df_reddit_demographics_normalized_labels['label']
del df_reddit_demographics_normalized_labels['label']
dataset_reddit_demographics_plbrt=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_demographics_normalized_labels[df_reddit_demographics_normalized_labels['split']==k]) for k in {'train','test','validation'}})

In [309]:
@dataclass
class DataCollatorForPolls:
    tokenizer: None=None
    tokenizer_kwargs: None=None
    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        
        flattened_features = sum(flattened_features, [])
        pad_args=inspect.signature(self.tokenizer.pad).parameters.keys()
        batch = self.tokenizer.pad(flattened_features, **fc.project(self.tokenizer_kwargs,pad_args))
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.stack(labels).float() #### Allows one value per choice

        return batch

In [310]:
reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_reddit_demographics_plbrt = [reco]

In [311]:
reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="title",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_plbrt = [reco]

In [312]:
def show_one(example):
    print(f"Context: {example['inputs']}")
    print(f"  A - {example['inputs']} {example['choice_0']}")
    print(f"  B - {example['inputs']} {example['choice_1']}")

In [313]:
show_one(dataset_reddit_demographics_plbrt["train"][0])

Context: democrat: Do You Believe Your Congressional District is Gerrymandered?
  A - democrat: Do You Believe Your Congressional District is Gerrymandered? Yes 
  B - democrat: Do You Believe Your Congressional District is Gerrymandered? No 


## WVMCQ7

### Filtering out non relevant questions

From the questionnaires we discard questions that do not serve to capture the opinion of people. Questions about family situation, sex, age etc of a demographic is not useable for our purpose.

In [314]:
dict_questions_discard = ["Religious person",
"Neighbours: Drug addicts",
"Neighbours: People of a different race",
"Neighbours: Immigrants/foreign workers",
"Neighbours: Homosexuals",
"Neighbours: Heavy drinkers",
"Sex",
"Year of birth",
"Age",
"Respondent immigrant",
"Mother immigrant",
"Father immigrant",
"Country of birth: Respondent",
"Country of birth: Mother of the respondent",
"Country of birth: Father of the respondent",
"Citizen of [country]",
"Number of people in household",
"Do you live with your parents",
"Marital status",
"How many children do you have",
"Highest educational level attained - Respondent",
"Education level (recoded)",
"Highest educational level attained - Respondent´s Spouse",
"Highest educational level attained - Respondent´s Spouse (Recoded)",
"Highest educational level attained - Respondent´s Mother",
"Highest educational level attained - Respondent´s Mother (Recoded)",
"Highest educational level attained - Respondent´s Father",
"Highest educational level attained - Respondent´s Father (Recoded)",
"Employment status",
"Pre-school child suffers with working mother",
"Employment status - Responent´s Spouse",
"Respondent - Occupational group",
"Respondent´s Spouse - Occupational group",
"Respondent´s Father - Occupational group (when respondent was 14 years old)",
"Sector of employment",
"Scale of incomes",
"State of health (subjective)",
]

In [315]:
def discard_questions(x):
    if(x['input'] in dict_questions_discard):
        pass
    else:
        return x

In [316]:
df_filtered = df.progress_apply(discard_questions,axis= 1)
df_filtered = df_filtered.dropna()

  0%|          | 0/110901 [00:00<?, ?it/s]

In [317]:
df_filtered = df_filtered.progress_apply(filter_data,axis=1)
df_filtered = df_filtered.dropna()
pd.DataFrame(df_filtered)

  0%|          | 0/103602 [00:00<?, ?it/s]

,year,demographic,input,demographic_topic,options,labels
0,2017.0,-1,Aims of country: first choice,age,"[A high level of economic growth, Making sure ...","[0.25, 0.25, 0.25, 0.25]"
1,2017.0,-1,Aims of country: second choice,age,[Making sure this country has strong defense f...,"[0.2, 0.8]"
2,2017.0,-1,Aims of respondent: first choice,age,"[maintaining order in the nation, Giving peopl...","[0.6000000000000001, 0.2, 0.2]"
3,2017.0,-1,Aims of respondent: second choice,age,[Giving people more say in important governmen...,"[0.5, 0.25, 0.25]"
4,2017.0,-1,Believe in: God,age,"[No, Yes]","[0.5, 0.5]"
...,...,...,...,...,...,...
209537,2022.0,male,Vote in elections: local level,gender,"[Always, Usually, Never, Not allowed to vote]","[0.5963541667000001, 0.24804687500000003, 0.14..."
209538,2022.0,male,Which party would you vote for: first choice (...,gender,"[No right to vote, Other, Independent Candidate]","[0.36250000000000004, 0.125, 0.5125000000000001]"
209539,2022.0,male,Willingness to fight for country,gender,"[no, yes]","[0.22697899840000002, 0.7730210016]"
209540,2022.0,male,Work is a duty towards society,gender,"[Strongly agree, Agree, Neither agree nor disa...","[0.3264913406, 0.3688261706, 0.1610006414, 0.1..."


In [318]:
df_filtered= pd.DataFrame([item for item in df_filtered.values if item[1].lower() not in ["-5","-4","-3","-2","-1","no answer","don't know","missing","other"] ],columns= ["year","demographic","input","demographic_category","options","labels"])

In [319]:
df_filtered = df_filtered[df_filtered["year"]>2019]

In [320]:
df_filtered['split']=df_filtered.input.progress_map(lambda x: get_split(x,0.80,0.20))

  0%|          | 0/39731 [00:00<?, ?it/s]

### Undersampling for dataset demographic attribute balancing on training set

In [321]:
newdf = df_filtered.sample(frac=1).reset_index(drop=True).drop_duplicates(
  subset = ['input', 'demographic'],
  keep = 'last').reset_index(drop = True)

In [322]:
df_filtered_undersampled = newdf.undersample(column="demographic")

In [323]:
df_filtered_undersampled.progress_apply(shuffle_options,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

,year,demographic,input,demographic_category,options,labels,split
0,2020.0,16,Feeling of happiness,age,"[very happy, not very happy, Rather happy]","[0.5714285714, 0.1071428571, 0.32142857140000003]",train
1,2020.0,18,"Member: Self-help group, mutual aid group",age,"[not mentioned, mentioned]","[0.796460177, 0.203539823]",train
2,2020.0,61,Scale of incomes (WVS7),age,"[Highest group, Lowest group]","[0.35000000000000003, 0.65]",validation
3,2022.0,82,"How close you feel: Continent; e.g. Europe, As...",age,"[not close at all, very close, close, not very...","[0.0952380952, 0.1904761905, 0.460317460300000...",train
4,2021.0,39,Institution of occupation,age,"[Private non-profit organization , Private bus...","[0.1074380165, 0.5785123967, 0.3140495868]",train
...,...,...,...,...,...,...,...
15223,2022.0,48,Employment status Respondent,age,"[No, no paid employment:Housewife not otherwis...","[0.0322580645, 0.0322580645, 0.016129032300000...",validation
15224,2021.0,Armenia,Democraticness in own country,country,"[Not at all democratic, Completely democratic]","[0.6115417743, 0.3884582257]",validation
15225,2022.0,26,Trust: Your neighborhood (B),age,"[do not trust at all, do not trust very much, ...","[0.027027027000000002, 0.24324324320000001, 0....",train
15226,2020.0,Myanmar,Important child qualities: good manners,country,"[mentioned, not mentioned]","[0.655, 0.34500000000000003]",train


### Add choice columns to dataset

In [324]:
df=df_filtered_undersampled.progress_apply(choices,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

In [325]:
if mode=='binary':
    df=df[df.labels<num_choices]
else:
    df=df[df.labels.map(len)==num_choices]
len(df)

15228

### Apply Templates

#### Zeroshot Setting for Masked Language Task


In [405]:
df_filtered_undersampled

,year,demographic,input,demographic_category,options,labels,split
0,2020.0,16,Feeling of happiness,age,"[very happy, Rather happy, not very happy]","[0.5714285714, 0.32142857140000003, 0.1071428571]",train
1,2020.0,18,"Member: Self-help group, mutual aid group",age,"[not mentioned, mentioned]","[0.796460177, 0.203539823]",train
2,2020.0,61,Scale of incomes (WVS7),age,"[Highest group, Lowest group]","[0.35000000000000003, 0.65]",validation
3,2022.0,82,"How close you feel: Continent; e.g. Europe, As...",age,"[very close, close, not very close, not close ...","[0.1904761905, 0.46031746030000004, 0.25396825...",train
4,2021.0,39,Institution of occupation,age,"[Government or public institution , Private bu...","[0.3140495868, 0.5785123967, 0.1074380165]",train
...,...,...,...,...,...,...,...
15223,2022.0,48,Employment status Respondent,age,"[Yes, has paid employment: Full time employee,...","[0.6451612903, 0.25806451610000003, 0.03225806...",validation
15224,2021.0,Armenia,Democraticness in own country,country,"[Completely democratic, Not at all democratic]","[0.3884582257, 0.6115417743]",validation
15225,2022.0,26,Trust: Your neighborhood (B),age,"[trust completely, trust somewhat, do not trus...","[0.1351351351, 0.5945945946, 0.243243243200000...",train
15226,2020.0,Myanmar,Important child qualities: good manners,country,"[not mentioned, mentioned]","[0.34500000000000003, 0.655]",train


In [326]:
year_index = 0
demographic_index= 1
input_index = 2
category_index = 3
options_index = 4
labels_index = 5
split_index = 6


dataset_wvs_sentences = []
for input in df_filtered_undersampled.values:
    year = input[year_index]
    demographic = input[demographic_index]
    question_input = str(input[input_index])
    category = input[category_index]
    options = input[options_index]
    labels = input[labels_index]
    split = input[split_index]

    for i in range(len(options)):
        label = labels[i]
        choice = options[i]
        sentence = f"In {int(year)}, <mask> percent of {demographic} voted '{choice}' on the subject '{question_input}'".lower()
        dataset_wvs_sentences.append({"year":year,"demographic":demographic,"subject":question_input,"demographic_category":category,"choice":choice,"input": sentence, "y": label,"split":split})
dataset_wvs_sentences = pd.DataFrame(dataset_wvs_sentences)

In [422]:
df[df["split"] == "validation"]

,year,demographic,input,demographic_category,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,question
2,2020.0,61,Scale of incomes (WVS7),age,"[0.35000000000000003, 0.65, -1.0, -1.0, -1.0, ...",validation,Highest group,Lowest group,,,,,61 years old: which of these options would you...
17,2021.0,32,Democraticness in own country,age,"[0.5730337079, 0.42696629210000003, -1.0, -1.0...",validation,Not at all democratic,Completely democratic,,,,,32 years old: which of these options would you...
23,2021.0,Kenya,Vote in elections: local level,country,"[0.5538707103, 0.2705506784, 0.166001596200000...",validation,Always,Usually,Never,Not allowed to vote,,,kenya: which of these options would you take i...
25,2020.0,53,Believe in: life after death,age,"[0.36571428570000003, 0.6342857143, -1.0, -1.0...",validation,No,Yes,,,,,53 years old: which of these options would you...
29,2022.0,40,Competition good or harmful,age,"[0.7592592593, 0.2407407407, -1.0, -1.0, -1.0,...",validation,Competition is good,Competition is harmful,,,,,40 years old: which of these options would you...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15206,2022.0,"No, no paid employment:Unemployed",How important is God in your life,employement,"[0.35338345860000003, 0.6466165414, -1.0, -1.0...",validation,not at all important,very important,,,,,"no, no paid employment:unemployed: how importa..."
15210,2020.0,Guatemala,"Member: Belong to conservation, the environmen...",country,"[0.7365771812, 0.2634228188, -1.0, -1.0, -1.0,...",validation,not mentioned,mentioned,,,,,guatemala: which of these options would you ta...
15216,2020.0,21,Justifiable: Political violence,age,"[0.9043659044000001, 0.09563409560000001, -1.0...",validation,Never justifiable,Always justifiable,,,,,21 years old: which of these options would you...
15223,2022.0,48,Employment status Respondent,age,"[0.6451612903, 0.25806451610000003, 0.03225806...",validation,"Yes, has paid employment: Full time employee","Yes, has paid employment: Part time employee","Yes, has paid employment: Self employed","No, no paid employment:Housewife not otherwise...","No, no paid employment:Student","No, no paid employment:Unemployed",48 years old: which of these options would you...


In [327]:
dataset_wvs_sentences['labels']=dataset_wvs_sentences['y']
try:
    del dataset_wvs_sentences['options']
except:
    pass
dataset_wvs_zeroshot=DatasetDict({k:datasets.Dataset.from_pandas(dataset_wvs_sentences[dataset_wvs_sentences['split']==k]) for k in {'train','test','validation'}})

### Zeroshot and fine-tuning setting for MCQ task

In [328]:
df = df.progress_apply(apply_question_template,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

### Training, Validation and Test Sets

In [331]:
df[df.labels.progress_map(lambda x: any(np.isnan(a) for a in x))]

  0%|          | 0/15228 [00:00<?, ?it/s]

,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,question


In [338]:
try:
    del df['options']
except:
    pass
dataset=DatasetDict({k:datasets.Dataset.from_pandas(df[df['split']==k]) for k in {'train','test','validation'}})

In [399]:
Counter(dataset_wvs_zeroshot["validation"]["subject"])

Counter({'Scale of incomes (WVS7)': 204,
         'Democraticness in own country': 204,
         'Vote in elections: local level': 373,
         'Believe in: life after death': 204,
         'Competition good or harmful': 204,
         'Jobs scarce: Men should have more right to a job than women\n(5-point scale)': 528,
         'Believe in: heaven': 208,
         'Member: Belong to education, arts, music or cultural activities': 210,
         'Future changes: less importance placed on work': 303,
         'Important child qualities: religious faith': 206,
         'Believe in: hell': 208,
         'Member: Belong to labour unions': 208,
         'How important is God in your life': 202,
         'Employment status Spouse/partner': 374,
         "How often in country's elections: Rich people buy elections": 419,
         'Mother immigrant / born in country\n': 190,
         'Duty towards society to have children': 513,
         'Aims of country: first choice': 419,
         'Democracy: 

In [404]:
dataset["validation"]["question"]

["61 years old: which of these options would you take in regard to the subject 'scale of incomes (wvs7)'?",
 "32 years old: which of these options would you take in regard to the subject 'democraticness in own country'?",
 "kenya: which of these options would you take in regard to the subject 'vote in elections: local level'?",
 "53 years old: which of these options would you take in regard to the subject 'believe in: life after death'?",
 "40 years old: which of these options would you take in regard to the subject 'competition good or harmful'?",
 "30 years old: which of these options would you take in regard to the subject 'democraticness in own country'?",
 "51 years old: which of these options would you take in regard to the subject 'jobs scarce: men should have more right to a job than women\n(5-point scale)'?",
 "26 years old: which of these options would you take in regard to the subject 'believe in: heaven'?",
 "34 years old: which of these options would you take in regard to 

In [333]:
def show_one(example):
    print(f"Context: {example['question']}")
    print(f"  A - {example['question']} {example['choice_0']}")
    print(f"  B - {example['question']} {example['choice_1']}")
    print(f"  C - {example['question']} {example['choice_2']}")
    print(f"  D - {example['question']} {example['choice_3']}")
    print(f"  D - {example['question']} {example['choice_4']}")

In [334]:
show_one(dataset["train"][8])

Context: singapore: which of these options would you take in regard to the subject 'government has the right: collect information about anyone living in [country] without their knowledge'?
  A - singapore: which of these options would you take in regard to the subject 'government has the right: collect information about anyone living in [country] without their knowledge'? Definitely should have the right
  B - singapore: which of these options would you take in regard to the subject 'government has the right: collect information about anyone living in [country] without their knowledge'? Probably should have the right
  C - singapore: which of these options would you take in regard to the subject 'government has the right: collect information about anyone living in [country] without their knowledge'? Probably should not have the right
  D - singapore: which of these options would you take in regard to the subject 'government has the right: collect information about anyone living in [cou

In [335]:
reco = tn.MultipleChoice(
    dataset,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_wvs = [reco]

## PollBERT Model

### insights

In [356]:
input_ids = torch.tensor([[   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2,  134,  111,  158,    2,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], [   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2, 1225,  111,  389,    2,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], [   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2, 2983,  111,   70,    9,  106,    2,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], [   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], [   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], [   0,  281,  811,   35, 1336,  171, 3102,  749,  115,   47,  766,  116,
           2,    2,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1]])

In [383]:
input_ids.size()

torch.Size([6, 256])

In [387]:
input_ids.view(-1, input_ids.size(-1)).size()

torch.Size([6, 256])

### Forward method

In [391]:
from transformers.models.roberta.modeling_roberta import *
  
        
def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_type_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], MultipleChoiceModelOutput]:
    r"""
    labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
        Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
        num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
        `input_ids` above)
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]

    flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
    flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
    flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
    flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
    flat_inputs_embeds = (
        inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
        if inputs_embeds is not None
        else None
    )

    outputs = self.roberta(
        flat_input_ids,
        position_ids=flat_position_ids,
        token_type_ids=flat_token_type_ids,
        attention_mask=flat_attention_mask,
        head_mask=head_mask,
        inputs_embeds=flat_inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    pooled_output = outputs[1]
    
    pooled_output = self.dropout(pooled_output)
    
    logits = self.classifier(pooled_output)
    print(self.classifier.in_features,self.classifier.out_features,self.classifier.bias)
    reshaped_logits = logits.view(-1, num_choices)
    print(outputs[1].size(),pooled_output.size(),logits.size(),reshaped_logits.size()) 

    m=nn.Softmax(dim=1)
    reshaped_logits=m(reshaped_logits)
    loss = None
    if labels is not None:
        loss_fct = self.loss
        loss = loss_fct(reshaped_logits, labels)
    if not return_dict:
        output = (reshaped_logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output
    
    return MultipleChoiceModelOutput(
        loss=loss,
        logits=reshaped_logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )
from types import MethodType 

In [91]:
from statistics import mean
def compute_metrics_old(element):
    f_distance = lambda x,y: np.abs((x-y)).max(axis=0).mean()
    y_pred,y_true = element
    distance = f_distance(y_pred,y_true)
    return distance
    
def crop(element):
  y_pred,y_true = element
  mask = y_true == -1
  return (y_pred[~mask],y_true[~mask])


def compute_metrics(self,batch):
  meta = {"name": self.name, "size": len(batch.predictions), "index": self.index}
  distance = mean([compute_metrics_old(crop(element)) for element in zip(batch.predictions,batch.label_ids)])
  return {**meta, 'distance':distance}


def mse_loss(input, target, ignored_index=-1):
    mask = target == ignored_index
    out = (input[~mask]-target[~mask])**2
    return out.mean()**.5

def bce_loss(input, target, ignored_index=-1):
  loss = nn.BCELoss()
  mask = target == ignored_index
  out = loss(input[~mask],target[~mask])
  return out

# Experiments

## Zeroshot

### Loading model

In [423]:
classifier = transformers.pipeline("fill-mask",device=0,model="roberta-base")

AssertionError: Torch not compiled with CUDA enabled

### Evaluation

#### WVMCQ7

In [425]:
dataset_wvs_zeroshot

DatasetDict({
    train: Dataset({
        features: ['year', 'demographic', 'subject', 'demographic_category', 'choice', 'input', 'y', 'split', 'labels', '__index_level_0__'],
        num_rows: 38272
    })
    test: Dataset({
        features: ['year', 'demographic', 'subject', 'demographic_category', 'choice', 'input', 'y', 'split', 'labels', '__index_level_0__'],
        num_rows: 0
    })
    validation: Dataset({
        features: ['year', 'demographic', 'subject', 'demographic_category', 'choice', 'input', 'y', 'split', 'labels', '__index_level_0__'],
        num_rows: 9819
    })
})

In [ ]:
results_wvs_bert = []
for example in dataset_wvs_zeroshot["validation"]:
  prediction = round(Decimal(int(classifier(example["input"])[0]["token_str"]) / 100),2)
  data = {"input":example["input"],"subject":example["subject"],"choice":example["choice"],"demographic":example["demographic"],"year":example["year"],"predictions":prediction,"ground_truth":example["y"]}
  results_wvs_bert.append(data)
pd.DataFrame(results_wvs_bert).to_csv("./validation_sets/roberta_wvs_results_validation.csv")

#### RedditPoll350k

# Finetuning

### Reddit Demo on Roberta

In [392]:
class args:
    model_name = "roberta-base"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps = 1
    

models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [393]:
trainer.train()

c:\Users\Quent\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11037
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 1
  Number of trainable parameters = 124646401


  0%|          | 0/1 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


768 1 Parameter containing:
tensor([0.], requires_grad=True)
torch.Size([6, 768]) torch.Size([6, 768]) torch.Size([6, 1]) torch.Size([1, 6])
768 1 Parameter containing:
tensor([0.], requires_grad=True)
torch.Size([6, 768]) torch.Size([6, 768]) torch.Size([6, 1]) torch.Size([1, 6])


KeyboardInterrupt: 

### WVS wihtout Reddit

In [ ]:
class args:
    model_name = "roberta-base"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps = 100
    

models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



 ------------------------------ START 1e-05 & 3 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.100900,0.084520,,208,0,0.170294
1,0.079100,0.094514,,208,0,0.191019
2,0.073900,0.115245,,208,0,0.203698


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5898, training_loss=0.08463457068090481, metrics={'train_runtime': 5138.5809, 'train_samples_per_second': 9.185, 'train_steps_per_second': 1.148, 'total_flos': 3.724389567347098e+16, 'train_loss': 0.08463457068090481, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.12094242870807648,
 'eval_name': '',
 'eval_size': 208,
 'eval_index': 0,
 'eval_distance': 0.20369841158390045,
 'eval_runtime': 12.0254,
 'eval_samples_per_second': 17.297,
 'eval_steps_per_second': 2.162,
 'epoch': 3.0,
 'Accuracy': nan}

In [ ]:
torch.cuda.empty_cache()

### WVS with Reddit finetuned (Full)

In [ ]:
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=4
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

In [ ]:
trainer.model.task_models_list[0].push_to_hub("Qubix/RoBERTa-WVSMCQ7")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-WVSMCQ7/commit/33c1374113124445e6168c49527f0b084f70ca26', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='33c1374113124445e6168c49527f0b084f70ca26', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
class args:
    model_name = "Qubix/RoBERTa-WVSMCQ7"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=4
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class args:
    model_name = "Qubix/RoBERTa-WVSMCQ7"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2991941273212433,
 'eval_name': '',
 'eval_size': 209,
 'eval_index': 0,
 'eval_distance': 0.34720391035079956,
 'eval_runtime': 11.6235,
 'eval_samples_per_second': 17.981,
 'eval_steps_per_second': 2.323,
 'Accuracy': nan}

In [ ]:
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.3640327751636505,
 'eval_name': '',
 'eval_size': 209,
 'eval_index': 0,
 'eval_distance': 0.5242656469345093,
 'eval_runtime': 14.946,
 'eval_samples_per_second': 13.984,
 'eval_steps_per_second': 1.807,
 'Accuracy': nan}

### Reddit with Reddit finetuned (Full)

#### Hyperparameters GRID SEARCH

~ 2:20:40 for first combination

In [ ]:
lr= 1e-4
bs = 4
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 0.0001 & 4 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.389500,0.368831,,254,0,0.533278
1,0.390000,0.380050,,254,0,0.533278
2,0.385900,0.366642,,254,0,0.533278


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=8277, training_loss=0.38846958958182315, metrics={'train_runtime': 8440.8129, 'train_samples_per_second': 3.923, 'train_steps_per_second': 0.981, 'total_flos': 2.613324215745331e+16, 'train_loss': 0.38846958958182315, 'epoch': 3.0})

~  1:11:54 for second combination

In [ ]:
lr= 1e-4
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 0.0001 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.389700,0.378001,,254,0,0.533278
1,0.388600,0.369769,,254,0,0.533278
2,0.344200,0.255908,,254,0,0.279023


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4137, training_loss=0.3741645102976946, metrics={'train_runtime': 4315.023, 'train_samples_per_second': 7.673, 'train_steps_per_second': 0.959, 'total_flos': 2.6123770159570944e+16, 'train_loss': 0.3741645102976946, 'epoch': 3.0})

~ 53:47 for third combination

In [ ]:
lr= 1e-4
bs = 16
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 0.0001 & 16 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.269700,0.227759,,254,0,0.254176
1,0.239700,0.210592,,254,0,0.241543
2,0.233900,0.230469,,254,0,0.242492


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=1032, training_loss=0.24776996198550674, metrics={'train_runtime': 3230.4743, 'train_samples_per_second': 10.25, 'train_steps_per_second': 0.319, 'total_flos': 2.606535950596301e+16, 'train_loss': 0.24776996198550674, 'epoch': 2.99})

~ for  2:19:44 fourth combination

In [ ]:
lr= 1e-5
bs = 4
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 4 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.247400,0.195566,,254,0,0.244280
1,0.218100,0.220026,,254,0,0.242190
2,0.217200,0.196327,,254,0,0.240249


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=8277, training_loss=0.227559931110872, metrics={'train_runtime': 8384.4201, 'train_samples_per_second': 3.949, 'train_steps_per_second': 0.987, 'total_flos': 2.613324215745331e+16, 'train_loss': 0.227559931110872, 'epoch': 3.0})

~1:11:03 for fifth combination

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.244200,0.202886,,254,0,0.242256
1,0.216600,0.197489,,254,0,0.240204
2,0.215600,0.207006,,254,0,0.238804


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4137, training_loss=0.22548083369213803, metrics={'train_runtime': 4264.6217, 'train_samples_per_second': 7.764, 'train_steps_per_second': 0.97, 'total_flos': 2.6123770159570944e+16, 'train_loss': 0.22548083369213803, 'epoch': 3.0})

~53:02 for sixth combination

In [ ]:
lr= 1e-5
bs = 16
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 16 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.277000,0.223006,,254,0,0.247216
1,0.235600,0.212097,,254,0,0.243471
2,0.233400,0.226280,,254,0,0.242173


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=1032, training_loss=0.24866703684015792, metrics={'train_runtime': 3185.4431, 'train_samples_per_second': 10.394, 'train_steps_per_second': 0.324, 'total_flos': 2.606535950596301e+16, 'train_loss': 0.24866703684015792, 'epoch': 2.99})

~ for seventh combination

In [ ]:
lr= 1e-6
bs = 4
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-06 & 4 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.308800,0.214850,,254,0,0.288970
1,0.254600,0.230952,,254,0,0.259844
2,0.239000,0.212346,,254,0,0.257870


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=8277, training_loss=0.2674744824820474, metrics={'train_runtime': 8405.8731, 'train_samples_per_second': 3.939, 'train_steps_per_second': 0.985, 'total_flos': 2.613324215745331e+16, 'train_loss': 0.2674744824820474, 'epoch': 3.0})

~ for eigthest combination

In [ ]:
lr= 1e-6
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-06 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.318900,0.255546,,254,0,0.342961
1,0.274300,0.204400,,254,0,0.274395
2,0.251700,0.243024,,254,0,0.267972


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4137, training_loss=0.28163181274502586, metrics={'train_runtime': 3722.1764, 'train_samples_per_second': 8.896, 'train_steps_per_second': 1.111, 'total_flos': 2.6123770159570944e+16, 'train_loss': 0.28163181274502586, 'epoch': 3.0})

~ for nineth combination

In [ ]:
lr= 1e-6
bs = 16
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=3
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-06 & 16 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.349700,0.316913,,254,0,0.403016
1,0.324200,0.285785,,254,0,0.377293
2,0.313800,0.306084,,254,0,0.369115


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=1032, training_loss=0.3292279428289842, metrics={'train_runtime': 2771.4996, 'train_samples_per_second': 11.947, 'train_steps_per_second': 0.372, 'total_flos': 2.606535950596301e+16, 'train_loss': 0.3292279428289842, 'epoch': 2.99})

#### other grid search

In [ ]:
lrs = [1e-4,1e-5,1e-6]#
bsz = [4,8,16]#
grid_search = {1e-4:{4:0,8:0,16:0},1e-5:{4:0,8:0,16:0},1e-6:{4:0,8:0,16:0}}
for lr in lrs:
  for bs in bsz:
    class args:
        model_name = "sileod/plbrt"
        # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
        learning_rate = lr
        num_train_epochs=3
        per_device_train_batch_size=bs
        per_device_eval_batch_size=bs
        gradient_accumulation_steps = bs
        models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
        max_steps=1

    models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
    models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
    tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
    trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
    print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
    trainOutput = trainer.train()
    grid_search[lr][bs] = trainOutput[1]   


In [ ]:
trainer.model.task_models_list[0].push_to_hub("Qubix/demoPollBERT")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/demoPollBERT/commit/68462075262ba681e088cf134191a28a5f573eab', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='68462075262ba681e088cf134191a28a5f573eab', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
torch.cuda.empty_cache()

#### Fine-Tuning

In [ ]:
lr= 3e-5
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default


In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.36841848492622375,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.5276154279708862,
 'eval_runtime': 25.473,
 'eval_samples_per_second': 9.971,
 'eval_steps_per_second': 1.256,
 'Accuracy': nan}

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 3e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.255700,0.217722,,254,0,0.251661
1,0.224500,0.260339,,254,0,0.244517
2,0.217600,0.200251,,254,0,0.241439
3,0.215000,0.158986,,254,0,0.238043


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.22821001214683703, metrics={'train_runtime': 5592.4079, 'train_samples_per_second': 7.894, 'train_steps_per_second': 0.986, 'total_flos': 3.483169354609459e+16, 'train_loss': 0.22821001214683703, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub("Qubix/RoBERTa-PollBERT350K_full_model")
trainer.model.task_models_list[0].push_to_hub("Qubix/RoBERTa-PollBERT350K")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/7.49k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-PollBERT350K/commit/9875a445781c7ad470a5772bcf77787193f001ed', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='9875a445781c7ad470a5772bcf77787193f001ed', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.235000,0.216306,,254,0,0.242761
1,0.216700,0.255326,,254,0,0.240703
2,0.216300,0.200082,,254,0,0.239873
3,0.214600,0.165904,,254,0,0.239016
4,0.212700,0.236502,,254,0,0.241479
5,0.210200,0.207509,,254,0,0.238653
6,0.208000,0.251372,,254,0,0.239867
8,0.206800,0.216043,,254,0,0.242528
8,0.204900,0.245425,,254,0,0.244471
9,0.204300,0.201877,,254,0,0.243279


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

TrainOutput(global_step=13790, training_loss=0.21295307427752788, metrics={'train_runtime': 13916.0006, 'train_samples_per_second': 7.931, 'train_steps_per_second': 0.991, 'total_flos': 8.707923386523648e+16, 'train_loss': 0.21295307427752788, 'epoch': 10.0})

### Eval

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-PollBERT350K"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=10
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True))

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.21321538090705872,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23804275691509247,
 'eval_runtime': 25.3863,
 'eval_samples_per_second': 10.005,
 'eval_steps_per_second': 1.261,
 'Accuracy': nan}

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-PollBERT350K"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=10
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_plbrt[0])
trainer = tn.Trainer(models, tasks_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True))

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.20897020399570465,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23723392188549042,
 'eval_runtime': 25.5036,
 'eval_samples_per_second': 9.959,
 'eval_steps_per_second': 1.255,
 'Accuracy': nan}

### WVS with Reddit finetuned (Full + Demographics)

In [ ]:
class args:
    model_name = "Qubix/demoPollBERT"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8

models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['torch.nn.BCELoss()','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

# Results

## Zeroshot

## RoBERTa

#### WVMCQ7

In [ ]:
results_wvs_bert = pd.read_csv("./validation_sets/roberta_wvs_results_validation.csv")
del results_wvs_bert['Unnamed: 0']
roberta_rmse_wvs = mean_squared_error(results_wvs_bert["predictions"],results_wvs_bert["ground_truth"]/100,squared=False)
roberta_rmse_wvs

In [ ]:
results_normalized = []
keys = list(pd.DataFrame(results_wvs_bert)[["subject","demographic","year"]].value_counts().keys())
for (subject,demographic,year) in keys:
    results = pd.DataFrame(results_wvs_bert).groupby(["subject","demographic","year"]).get_group((subject,demographic,year))
    for result in results.values:
        prob = round(Decimal(result[5]/sum(results["predictions"])),2)
        result[5] = prob
        results_normalized.append(result)
        
bert_wvs_normalized_results = pd.DataFrame(results_normalized,columns=["input","subject","choice","demographic","year","prediction","ground_truth"])
bert_wvs_normalized_results.to_csv("./validation_sets/roberta_wvs_normalized_results_validation.csv")

In [ ]:
bert_wvs_normalized_results = pd.read_csv("./validation_sets/roberta_wvs_normalized_results_validation.csv")
roberta_rmse_wvs_normalized = mean_squared_error(bert_wvs_normalized_results["prediction"],bert_wvs_normalized_results["ground_truth"]/100,squared=False)
roberta_rmse_wvs_normalized

#### Reddit (Full)

In [107]:
def rmse(x):
    preds = torch.tensor((x["zeroshot_preds"] +[0]*6)[:6])
    #(list(x.options.keys())+['']*num_choices)[:num_choices]
    x["rmse"] = mse_loss(preds,torch.tensor(x["labels"]))
    return x

In [108]:
results_reddit_bert = pd.read_json("./roberta_zeroshot_results_RedditPoll350k.json")

In [ ]:
results_reddit_bert = results_reddit_bert.progress_apply(rmse,axis=1)

In [110]:
np.mean([t.item() for t in results_reddit_bert["rmse"].values])

0.17329437242470014

#### RedditPoll350k (Demographics)

In [118]:
results_reddit_bert = pd.read_csv("D:/OneDrive/User/Documents/School/KU_Leuven/Master/2023/Unnused/old_zeroshot_results/validation_sets/roberta_wvs_normalized_results_validation.csv")

In [ ]:
results_normalized = []
keys = list(pd.DataFrame(results_reddit_bert)[["subject","demographic","year"]].value_counts().keys())
for (subject,demographic,year) in keys:
    results = pd.DataFrame(results_reddit_bert).groupby(["subject","demographic","year"]).get_group((subject,demographic,year))
    for result in results.values:
        prob = (result[6]/sum(results["predictions"]))
        result[6] = prob
        results_normalized.append(result)
        
bert_reddit_normalized_results = pd.DataFrame(results_normalized,columns=["index","input","subject","choice","demographic","year","prediction","ground_truth"])
del bert_reddit_normalized_results["index"]
bert_reddit_normalized_results.to_csv("./validation_sets/roberta_reddit_normalized_results_validation.csv")

In [119]:
bert_reddit_normalized_results = pd.read_csv("D:/OneDrive/User/Documents/School/KU_Leuven/Master/2023/Unnused/old_zeroshot_results/validation_sets/roberta_reddit_normalized_results_validation.csv")

In [120]:
from sklearn.metrics import mean_squared_error
roberta_rmse_reddit_demographics_normalized = mean_squared_error(bert_reddit_normalized_results["prediction"],bert_reddit_normalized_results["ground_truth"],squared=False)
roberta_rmse_reddit_demographics_normalized

0.3072572218642799

### Constant Predition Model

#### WVMCQ7

In [111]:
def constant_prediction(x):
    x["constant_preds"] = [1/len(x['zeroshot_preds'])]*len(x["labels"])
    return x

In [112]:
results_reddit_bert = pd.read_json("./roberta_zeroshot_results_RedditPoll350k.json")
results_reddit_bert = results_reddit_bert.progress_apply(constant_prediction,axis=1)

  0%|          | 0/7714 [00:00<?, ?it/s]

In [113]:
results_reddit_bert

,category,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,created,id,index,inputs,labels,level_0,n_answers,split,task,title,zeroshot_preds,constant_preds
0,diaries_&_daily_life,Yes,Only snap,Only whistle,No,IGN,IGN,2022,yur32f,14,Can you both snap your fingers and whistle?,"[0.6615384615000001, 0.1923076923, 0.069230769...",0,130,validation,0,Can you both snap your fingers and whistle?,"[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25, 0.25, 0.25]"
1,film_tv_&_video,0,1,2,3,4 or more,IGN,2022,yucvxt,99,"If you had the same power as Thanos, how many ...","[0.504587156, 0.128440367, 0.03669724770000000...",1,109,validation,0,"If you had the same power as Thanos, how many ...","[0.2, 0.2, 0.2, 0.2, 0.2]","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2]"
2,diaries_&_daily_life,YouTuber,Twitch Streamer,Athlete,Actress,Singer,Teacher,2022,yu58u8,132,What type of Women do you prefer out of these ...,"[0.0651558074, 0.0481586402, 0.3172804533, 0.1...",2,353,validation,0,What type of Women do you prefer out of these ...,"[0.1666666667, 0.1666666667, 0.1666666667, 0.1...","[0.16666666666666666, 0.16666666666666666, 0.1..."
3,news_&_social_concern,Yes,"No, they are separate from Asians",I'm Indian/Asian,I'm Indian and consider us different from Asians.,IGN,IGN,2022,yu3hva,145,Are Indians (Country) Asians in your book?,"[0.8070175439, 0.1666666667, 0.026315789500000...",3,114,validation,0,Are Indians (Country) Asians in your book?,"[0.2537497188, 0.2462502812, 0.2462502812, 0.2...","[0.25, 0.25, 0.25, 0.25, 0.25, 0.25]"
4,food_&_dining,“Mother’s cookies”,Those hard vanilla cookies with the stiff fros...,Results/what are those cookies?,IGN,IGN,IGN,2022,ytuk6v,196,Which cookie are you nostalgic about?,"[0.1935483871, 0.1935483871, 0.6129032258, -1....",4,62,validation,0,Which cookie are you nostalgic about?,"[0.3366498354, 0.3366498354, 0.32670032920000003]","[0.3333333333333333, 0.3333333333333333, 0.333..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7718,film_tv_&_video,Awesome,Meh!,Good,Bad!,IGN,IGN,2020,frnq8r,312022,What do you think about King Arthur: Legend of...,"[0.2173913043, 0.347826087, 0.1304347826, 0.30...",7718,23,validation,0,What do you think about King Arthur: Legend of...,"[0.244417289, 0.2518609037, 0.2518609037, 0.25...","[0.25, 0.25, 0.25, 0.25, 0.25, 0.25]"
7719,news_&_social_concern,Pink,Purple,IGN,IGN,IGN,IGN,2020,frccus,312061,"Help us settle a quarantine argument, what col...","[0.7196652720000001, 0.28033472800000003, -1.0...",7719,239,validation,0,"Help us settle a quarantine argument, what col...","[0.5, 0.5]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"
7720,news_&_social_concern,"You see a bustling village ahead of you, and h...",The flora and fauna you see are utterly alien....,The land around you is barren. You think you c...,IGN,IGN,IGN,2020,fr04oe,312093,What-If Hypothetical Poll Series (#2) [VRMMO],"[0.3404255319, 0.3191489362, 0.3404255319, -1....",7720,47,validation,0,What-If Hypothetical Poll Series (#2) [VRMMO],"[0.33333333330000003, 0.33333333330000003, 0.3...","[0.3333333333333333, 0.3333333333333333, 0.333..."
7721,news_&_social_concern,Killed by powerful people,Actually did kill himself,IGN,IGN,IGN,IGN,2020,fqt3bw,312113,How do you think Epstein died?,"[0.8105263158, 0.18947368420000002, -1.0, -1.0...",7721,95,validation,0,How do you think Epstein died?,"[0.5, 0.5]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"


In [82]:
def rmse_constant(x):
    x["rmse"] = mse_loss(torch.tensor(x["constant_preds"]),torch.tensor(x["labels"]))
    return x

In [83]:
results_reddit_bert = results_reddit_bert.progress_apply(rmse_constant,axis=1)
np.mean([t.item() for t in results_reddit_bert["rmse"].values])

  0%|          | 0/7714 [00:00<?, ?it/s]

tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500]) tensor([ 0.6615,  0.1923,  0.0692,  0.0769, -1.0000, -1.0000]) tensor(0.2426)
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000]) tensor([ 0.5046,  0.1284,  0.0367,  0.0275,  0.3028, -1.0000]) tensor(0.1816)
tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]) tensor([0.0652, 0.0482, 0.3173, 0.1048, 0.1218, 0.3428]) tensor(0.1182)
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500]) tensor([ 0.8070,  0.1667,  0.0263,  0.0000, -1.0000, -1.0000]) tensor(0.3278)
tensor([0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333]) tensor([ 0.1935,  0.1935,  0.6129, -1.0000, -1.0000, -1.0000]) tensor(0.1977)
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000]) tensor([ 0.5616,  0.2877,  0.0274,  0.0959,  0.0274, -1.0000]) tensor(0.2044)
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000]) tensor([ 0.1433,  0.1033,  0.1529,  0.0592,  0.5413, -1.0000]) tensor(0.1738)
tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]) tens

0.17314072440134248

In [ ]:
def constant_prediction(row):
  subject = row["subject"]
  demographic = row["demographic"]
  year = row["year"]
  try:
    pred = round((1/len(results_wvs_bert.groupby(["subject","demographic","year"]).get_group((subject,demographic,year)))),2)
    row["predictions"] = pred
  except:
    pass
  return row

In [ ]:
results_wvs_bert = pd.read_csv("./validation_sets/roberta_wvs_results_validation.csv")
results_reddit_bert_full = pd.read_csv("./validation_sets/roberta_reddit_full_results_validation.csv")
results_reddit_bert_demographics= pd.read_csv("./validation_sets/roberta_reddit_results_validation.csv")

In [ ]:
results_wvs_random = results_wvs_bert.apply(constant_prediction,axis=1)
results_wvs_random.to_csv("./validation_sets/results_wvs_random_validation.csv")

results_wvs_random = pd.read_csv("./validation_sets/results_wvs_random_validation.csv")


In [ ]:
from sklearn.metrics import mean_squared_error
constant_rsme_wvs = mean_squared_error(results_wvs_random["predictions"],results_wvs_random["ground_truth"]/100,squared=False)
constant_rsme_wvs

#### RedditPoll350k

In [ ]:
results_reddit_full_random = results_reddit_bert_full.apply(constant_prediction,axis=1)

results_reddit_full_random.to_csv("./validation_sets/results_reddit_full_random_validation.csv")

results_reddit_full_random = pd.read_csv("./validation_sets/results_reddit_full_random_validation.csv")

In [ ]:
results_reddit_random = results_reddit_bert_demographics.apply(constant_prediction,axis=1)

results_reddit_random.to_csv("./validation_sets/results_reddit_random_validation.csv")

results_reddit_random = pd.read_csv("./validation_sets/results_reddit_random_validation.csv")

In [ ]:
constant_rsme_reddit_full = mean_squared_error(results_reddit_full_random["predictions"],results_reddit_full_random["ground_truth"],squared=False)
constant_rsme_reddit_full

In [ ]:
constant_rsme_reddit_demographics = mean_squared_error(results_reddit_random["predictions"],results_reddit_random["ground_truth"],squared=False)
constant_rsme_reddit_demographics

## Fine tuning